# Filtered Shortest Path

Finding all the shortest paths between two nodes, using a filtered set of properties.

*outstanding questions:*
- can we order/filter these paths by 'uniqueness'? (e.g. connecting people because they are humans isn't that interesting!)

In [1]:
# Imports
from rdflib import Graph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_graph
import networkx as nx
from networkx import Graph as NXGraph
from networkx.algorithms.traversal.beamsearch import bfs_beam_edges
from networkx.algorithms.shortest_paths.generic import all_shortest_paths
from itertools import islice
import statistics
import collections
from rdflib import URIRef, Literal
import requests
import json


In [2]:
url = "https://d0rgkq.deta.dev/labels"

def get_labels(entities):
    payload = json.dumps({
      "uris": entities
    })
    headers = {
      'Content-Type': 'application/json'
    }

    return requests.post(url, headers=headers, data=payload).json()


In [3]:
# RDF graph loading
# This takes a while (10+ minutes). If you're working on a local machine it'll 
# be better to download the file from `path` below and give this notebook a 
# local path.
path = "./hc_dump_latest-filtered-20211117-114506.nt"
rg = Graph()
rg.parse(path, format='nt')
print("rdflib Graph loaded successfully with {} triples".format(len(rg)))

rdflib Graph loaded successfully with 2030542 triples


In [4]:
# Conversion of rdflib.Graph to networkx.Graph
if 'subg' in locals():
  print("Using subgraph generated in last cell")
  G = rdflib_to_networkx_graph(subg)
else:
  print("Using entire rdf graph")
  G = rdflib_to_networkx_graph(rg)
print("networkx Graph loaded successfully with length {}".format(len(G)))

Using entire rdf graph
networkx Graph loaded successfully with length 758632


In [5]:
edges = {
    str(node): n_edges for node, n_edges in G.degree()
}

edges = {k: v for k, v in sorted(edges.items(), key=lambda item: item[1])}


## Shortest path

In [6]:
ent_a = URIRef("http://collections.vam.ac.uk/item/O1389838")
ent_b = URIRef("https://collection.sciencemuseumgroup.org.uk/objects/co102121")

In [7]:
all_sps = all_shortest_paths(G, ent_a, ent_b)
path_graphs = [nx.path_graph(sp) for sp in all_sps]

In [8]:
for idx, p in enumerate(path_graphs):
    print(f"Path {idx+1}")
    for idx, ea in enumerate(p.edges()):
        subj = ea[0]
        edges = [i[1] for i in G.edges[ea[0], ea[1]]['triples']]
        obj = ea[1]
        
        ent_labels = get_labels([e for e in ea if e.startswith("http")])

        if idx +1 < len(p.edges()):
            print(f"- {ent_labels.get(str(subj)) or subj} -> {edges[0]}")
        else:
            print(f"- {ent_labels.get(str(subj)) or subj} -> {edges[0]}")
            print(f"- {ent_labels.get(str(obj)) or obj}")


Path 1
- Mrs. Jordan as Widow Belmour -> http://www.heritageconnector.org/RDF/entityLOC
- England -> http://www.heritageconnector.org/RDF/entityLOC
- Photocopied letter and original notes from Samuel Holmes, New York and Pacific Steamship, Coal and Lumbar Company, Broad Street, New York to Robert Young -> http://www.heritageconnector.org/RDF/entityORG
- Soho -> http://www.heritageconnector.org/RDF/entityLOC
- Ceramic pot lid for Bear's grease
Path 2
- Mrs. Jordan as Widow Belmour -> http://www.heritageconnector.org/RDF/entityLOC
- England -> http://www.heritageconnector.org/RDF/entityNORP
- Longcase clock -> http://www.heritageconnector.org/RDF/entityLOC
- Soho -> http://www.heritageconnector.org/RDF/entityLOC
- Ceramic pot lid for Bear's grease
Path 3
- Mrs. Jordan as Widow Belmour -> http://www.heritageconnector.org/RDF/entityLOC
- England -> http://www.heritageconnector.org/RDF/entityNORP
- Paul Crespin -> http://www.heritageconnector.org/RDF/entityLOC
- Soho -> http://www.heritagec